## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)

In [3]:
# Determine the number of unique values in each column.
print(application_df.nunique())

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [4]:
# Look at APPLICATION_TYPE value counts for binning
print(application_df['APPLICATION_TYPE'].value_counts())

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff = 500
application_counts = application_df['APPLICATION_TYPE'].value_counts()
application_types_to_replace = list(application_counts[application_counts < cutoff].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
print(application_df['CLASSIFICATION'].value_counts())

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C2561        1
C3700        1
C1728        1
C2150        1
C6100        1
Name: CLASSIFICATION, Length: 71, dtype: int64


In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts[classification_counts > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C6000       15
C1800       15
C1250       14
C8200       11
C1238       10
C1278       10
C1237        9
C1235        9
C7210        7
C4100        6
C1720        6
C2400        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C1234        2
C1256        2
C1246        2
C1267        2
C3200        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff = 1000
classification_counts = application_df['CLASSIFICATION'].value_counts()
classifications_to_replace = list(classification_counts[classification_counts < cutoff].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Look at INCOME_AMT value counts for binning
print(application_df['ASK_AMT'].value_counts())

5000       25398
10478          3
15583          3
6725           3
63981          3
           ...  
772556         1
70103          1
27096          1
25049          1
1138700        1
Name: ASK_AMT, Length: 8747, dtype: int64


In [11]:
# Create a dictionary to map income amount ranges to numerical values
income_map = {
    '0': 0,
    '1-9999': 1,
    '10000-24999': 2,
    '25000-99999': 3,
    '100000-499999': 4,
    '500000-999999': 5,
    '1M-5M': 6,
    '5M-10M': 7,
    '10M-50M': 8,
    '50M+': 9
}

# Replace the income amount ranges with numerical values
application_df['INCOME_AMT'] = application_df['INCOME_AMT'].replace(income_map)

# Print the unique values in the INCOME_AMT column
print(application_df['INCOME_AMT'].unique())

[0 1 2 4 8 3 9 6 7]


In [12]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df, columns=['AFFILIATION', 'USE_CASE', 'ORGANIZATION', 'SPECIAL_CONSIDERATIONS','APPLICATION_TYPE', 'CLASSIFICATION'])
application_df.head()

,STATUS,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,...,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C3000,CLASSIFICATION_Other
0,1,0,5000,1,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,1,1,108590,1,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,1,0,5000,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
3,1,2,6692,1,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,1,4,142590,1,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [20]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL'].values
X = application_df.drop('IS_SUCCESSFUL', axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

## Compile, Train and Evaluate the Model

In [29]:
# Define the model - deep neural net
nn = tf.keras.models.Sequential()

# Add input layer
nn.add(tf.keras.layers.Dense(units=35, input_dim=35, activation="relu"))

# Add hidden layers with dropout regularization
nn.add(tf.keras.layers.Dense(units=25, activation="relu"))
nn.add(tf.keras.layers.Dropout(0.2))
nn.add(tf.keras.layers.Dense(units=15, activation="relu"))
nn.add(tf.keras.layers.Dropout(0.2))

# Add output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 35)                1260      
                                                                 
 dense_4 (Dense)             (None, 25)                900       
                                                                 
 dropout (Dropout)           (None, 25)                0         
                                                                 
 dense_5 (Dense)             (None, 15)                390       
                                                                 
 dropout_1 (Dropout)         (None, 15)                0         
                                                                 
 dense_6 (Dense)             (None, 1)                 16        
                                                                 
Total params: 2,566
Trainable params: 2,566
Non-traina

In [30]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [33]:
# Import the ModelCheckpoint callback
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the callback to save the model weights every five epochs
checkpoint_path = "model_weights/checkpoint.ckpt"
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    save_freq=5*len(X_train_scaled),
    verbose=1
)
# Train the model with the callback
model = nn.fit(
    X_train_scaled, 
    y_train, 
    epochs=100, 
    batch_size=32, 
    validation_data=(X_test_scaled, y_test),
    callbacks=[cp_callback]
)

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5486 - accuracy: 0.7345 - val_loss: 0.5507 - val_accuracy: 0.7263
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5468 - accuracy: 0.7348 - val_loss: 0.5519 - val_accuracy: 0.7250
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5467 - accuracy: 0.7348 - val_loss: 0.5524 - val_accuracy: 0.7265
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5471 - accuracy: 0.7336 - val_loss: 0.5522 - val_accuracy: 0.7250
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5477 - accuracy: 0.7326 - val_loss: 0.5518 - val_accuracy: 0.7257
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5475 - accuracy: 0.7345 - val_loss: 0.5521 - val_accuracy: 0.7263
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5477 - accuracy: 0.7349 - val_loss: 0.5524 - val_accuracy: 0.7248

Epoch 58/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5467 - accuracy: 0.7358 - val_loss: 0.5520 - val_accuracy: 0.7266
Epoch 59/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5462 - accuracy: 0.7347 - val_loss: 0.5518 - val_accuracy: 0.7278
Epoch 60/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5472 - accuracy: 0.7357 - val_loss: 0.5525 - val_accuracy: 0.7278
Epoch 61/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5465 - accuracy: 0.7355 - val_loss: 0.5525 - val_accuracy: 0.7268
Epoch 62/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5465 - accuracy: 0.7348 - val_loss: 0.5526 - val_accuracy: 0.7268
Epoch 63/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5470 - accuracy: 0.7340 - val_loss: 0.5520 - val_accuracy: 0.7275
Epoch 64/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5460 - accuracy: 0.7346 - val_loss: 0.5529 - val_accuracy:

In [34]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5529 - accuracy: 0.7269 - 177ms/epoch - 660us/step
Loss: 0.5528959035873413, Accuracy: 0.7268804907798767


In [35]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity.h5")